In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

In [2]:
sitename = "nzd0161"
transect_id = "nzd0161-0135"
water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv")
water.index = pd.to_datetime(water.dates)
water = water[transect_id]
veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv")
veg.index = pd.to_datetime(veg.dates + "+00:00").dt.floor("s")
veg = veg[transect_id]
display((veg < water.mean()).value_counts())
veg = veg[veg < water.mean()]

nzd0161-0135
False    320
True      50
Name: count, dtype: int64

In [3]:
df = pd.DataFrame({"water": water, "veg": veg})
px.scatter(df, trendline="ols", title=transect_id)

In [4]:
from coastsat import SDS_transects

def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    if len(chainage) < 5:
        return chainage
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)

despiked = despike(veg)
smoothed = despiked.rolling("180d", min_periods=1).mean()
df = pd.DataFrame({"water": water, "raw": veg, "despiked": despiked, "smoothed": smoothed})
px.scatter(df, trendline="ols", title=transect_id)

In [5]:
shorelines = gpd.read_file("../CoastSat/shorelines.geojson")
shorelines = shorelines[shorelines.id.str.startswith("nzd")]
sites = sorted(shorelines.id.unique())

In [6]:
water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv").drop(columns="satname")
water.dates = pd.to_datetime(water.dates)
water.set_index("dates", inplace=True)
veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv")
veg.dates = pd.to_datetime(veg.dates + "+00:00").dt.floor("s")
veg.set_index("dates", inplace=True)
satname = veg.pop("satname")
veg = veg[veg.lt(water.rolling("30d", min_periods=1).mean())]
veg["satname"] = satname
veg

,nzd0161-0000,nzd0161-0001,nzd0161-0002,nzd0161-0003,nzd0161-0004,nzd0161-0005,nzd0161-0006,nzd0161-0007,nzd0161-0008,nzd0161-0009,...,nzd0161-0294,nzd0161-0295,nzd0161-0296,nzd0161-0209,nzd0161-0210,nzd0161-0211,nzd0161-0033,nzd0161-0034,nzd0161-0035,satname
dates,,,,,,,,,,,,,,,,,,,,,
1999-08-26 22:04:44+00:00,276.56,285.78,328.49,NaN,281.05,227.28,282.17,172.08,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L7
1999-11-14 22:04:42+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L7
2000-01-01 22:04:37+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L7
2000-02-18 22:04:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L7
2000-05-24 22:03:46+00:00,347.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-02 22:11:40+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L8
2025-03-02 22:12:04+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L8
2025-03-10 22:11:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L9


In [7]:
def process_site(sitename):
    water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv").drop(columns="satname")
    water.dates = pd.to_datetime(water.dates)
    water.set_index("dates", inplace=True)

    try:
        veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv")
    except:
        return
    veg.dates = pd.to_datetime(veg.dates + "+00:00").dt.floor("s")
    veg.set_index("dates", inplace=True)
    satname = veg.pop("satname")
    veg = veg[veg.lt(water.rolling("30d", min_periods=1).mean())]

    veg = veg.apply(despike, axis=0)

    for transect_id in veg.columns:
        veg[transect_id] = veg[transect_id].rolling("180d", min_periods=1).mean()

    veg["satname"] = satname
    veg.to_csv(f"Data/{sitename}/transect_time_series_smoothed.csv", index_label="dates")

process_map(
    process_site,
    shorelines.id.unique(),
    max_workers=32,
    chunksize=1,
    desc="Processing sites",
    position=0,
    leave=True,
)

Processing sites:   0%|          | 0/578 [00:00<?, ?it/s]

*

*

*

*

...

*

*

*

*

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,